# How to Use Eir Spatial Models
The static spatial models that Eir offers are the Hub and Strong Infectious models. This tutorial will use the Hub model as an example, but the same principle applies for the Strong Infectous model. The details about the model can be found in Fujie & Odagaki's paper, cited in the Eir JOSS paper. You could also find details in the docs/ folder. 

In [2]:
from Eir import HubSIR, HubSIRV # import the relevant Hub models that will be used in tutorial
import numpy as np # standard imports
import pandas as pd
from matplotlib import pyplot as plt

## Synopsis of how these spatial models work
Assume there are N separate individuals in a locality, confined to an LxL grid, with each person's location within the grid being randomly generated. These individuals do not change position, hence the "static" spatial model. 

To become infected, or leave the S compartment, we use a formula pulled from Fujie & Odagaki's paper. This formula is based on 3 variables: *r*, which represents the distance between the infectious person and susceptible person, *r0*, which is considered the "spreading radius" of the infectious person, where outside this radius, there is a 0% chance that the infectious person will spread it to the susceptible person outside the radius, and *w <sub>0* indicates the probability of an infectious person infecting a susceptible person 0 units away. It is better to simply look at the formula used in their article, as I still cannot figure out Markdown formatting. 

In the cell below, we initialize Hub model specific parameters that are used when generating the object.
   

In [3]:
pss = .17 # the probability of being a super spreader; has different definitions for Hub and Strong Infectious model. The differences of how super spreaders are treated can be found in documentation.
rstart = 3 # the distance of a normal spreader's spreading radius. In the Hub Model, this is scaled up by a factor of k for super spreaders.
side = 100 # the size of one side of the square plane that the individuals are confined to.
w0 = .93 # the w0 used in the formula; the probability an infectious person infects a susceptible person 0 units away
k = 6 ** 0.5 # the Hub constant that is used for scaling up rstart for super spreaders in the Hub model
alpha = 2 # another constant used in the spreading probability formula
days = 31 # the number of days being simulated

 

In the next cell, the parameters specific to the SIR model are initialized.

In [6]:
S0 = 9999 # initial number of susceptible individuals
I0 = 1 # initial number of infecteds
R0 = 0 # initial number of recovereds
gamma = .15 # the probability that an individual moves from I -> R. 

All the parameters have been initialized. The next step is to create a HubSIR object.

In [7]:
sim = HubSIR(pss=pss, rstart=rstart, side=side, w0=w0, hubConstant=k, alpha=alpha, days=days, S0=S0, I0=I0, R0=R0, gamma=gamma) # initialize the HubSIR object

When you run a simulation for a particular object, you use the run() method. This method has a default paremeter getDetails which is initialized to True. If getDetails=True, then the run() method returns a Simul_Details object, which can be used to delve deeper into the simulation. It will be featured later in this tutorial.

In [8]:
details = sim.run() # run the simulation and return a Simul_Details object

The simulation can also be retrieved as a pandas dataframe using the toDataFrame() method. It is important to do this only after the run() method has been called.

In [9]:
df = sim.toDataFrame()
df

,Days,Susceptible,Infected,Recovered
0,0.0,9999.0,1.0,0.0
1,1.0,9968.0,32.0,0.0
2,2.0,9819.0,175.0,6.0
3,3.0,9660.0,311.0,29.0
4,4.0,9449.0,477.0,74.0
5,5.0,9214.0,646.0,140.0
6,6.0,8951.0,818.0,231.0
7,7.0,8687.0,973.0,340.0
8,8.0,8405.0,1122.0,473.0
9,9.0,8021.0,1344.0,635.0


## Simul_Details object

Now, it is time to learn how to interact with the Simul_Details object. This object allows you to see a variety of things, from a particular person's state history to transmission chains. One thing you might want to do is find the state history of a particular person's number, such as the 4th person. You can simply use the personHistory() method and pass in the number person you want to view

In [10]:
stateOfNine = details.personHistory(9) # get the state history of person 9. Important to note that the people's numbers go from person 0, 1, 2... N-1, where N is the starting population of the simulation.
stateOfNine

[(0, 'S'), (14, 'I'), (20, 'R')]

From the above cell, we can see that for Person 9, they were susceptible on the 0th day. Then, on the 14th day, they changed to be infected. As you can see, it is very easy to read which state a given person was on what day and how they changed as the simulation progressed. 

Let's say that we also wanted to find the tranmission history of person 9; that is, who did they infect and on what day did they infect that person. For this, we'd use the personTransmissionHistory function and input the person number as an argument. 

In [11]:
transOfNine = details.personTransmissionHistory(9) # get the transmission history of person 9
transOfNine

[(7336, 16)]

The way to interpret the above output is that person 9 infected person 7336 on day 16. This list can be far more extensive and have multiple tuples, as can been in the following output:

In [19]:
transOfSeven = details.personTransmissionHistory(9999) # get the transmission history of person 999
transOfSeven

[(39, 1),
 (147, 1),
 (814, 1),
 (898, 1),
 (1088, 1),
 (1296, 1),
 (1714, 1),
 (2204, 1),
 (2249, 1),
 (2291, 1),
 (2546, 1),
 (2822, 1),
 (3278, 1),
 (3442, 1),
 (3773, 1),
 (4135, 1),
 (4302, 1),
 (4543, 1),
 (4753, 1),
 (4821, 1),
 (5104, 1),
 (5304, 1),
 (5490, 1),
 (5747, 1),
 (6819, 1),
 (6918, 1),
 (6986, 1),
 (8675, 1),
 (9106, 1),
 (9510, 1),
 (9805, 1),
 (3252, 2)]

You can also have an empty list, which means they didn't transmit the disease to anyone. 

If you want to get ALL of the transmissions of a particular day, then you can use the transmissionHistoryOnDay method and input the day as an argument.

In [20]:
transmission = details.transmissionHistoryOnDay(3) # get the entire transmission history on day 3. The days go from [1, days of simultion]
transmission 

[(218, 1260),
 (218, 8858),
 (240, 566),
 (240, 3352),
 (240, 3634),
 (240, 5511),
 (240, 6265),
 (365, 354),
 (365, 5533),
 (365, 7282),
 (365, 9323),
 (366, 2692),
 (366, 2819),
 (366, 4367),
 (366, 4447),
 (723, 3208),
 (775, 1878),
 (973, 4365),
 (1095, 281),
 (1095, 3902),
 (1095, 4598),
 (1095, 6832),
 (1172, 4400),
 (1236, 971),
 (1236, 1279),
 (1236, 1849),
 (1236, 2951),
 (1236, 3880),
 (1236, 5693),
 (1236, 6591),
 (1236, 6846),
 (1236, 7744),
 (1236, 8318),
 (1236, 9423),
 (1236, 9635),
 (1238, 472),
 (1238, 826),
 (1245, 299),
 (1245, 1504),
 (1245, 1860),
 (1245, 6565),
 (1245, 7388),
 (1245, 7465),
 (1245, 7928),
 (1245, 8206),
 (1354, 5973),
 (1393, 477),
 (1393, 1254),
 (1393, 4541),
 (1612, 262),
 (1612, 897),
 (1612, 1073),
 (1612, 2573),
 (1612, 2937),
 (1612, 3365),
 (1612, 3379),
 (1612, 3459),
 (1612, 3592),
 (1612, 6200),
 (1612, 6511),
 (1612, 6561),
 (1612, 7977),
 (1612, 8374),
 (1612, 9250),
 (1612, 9557),
 (1612, 9991),
 (1714, 2460),
 (1724, 2597),
 (2291, 

Each tuple represents a transmission event. The way to interpret it is the first numbered person infected the second number person. For example, for the first tuple in the list, person 218 infected person 1260 on day 3. 

Finally, if you want to examine how many transmissions every person has throughout the duration of the simulation, you can use the sortedTranmissions() method. This method returns a list of tuples that will return the number of tranmissions each person has, sorted from highest number of transmissions to lowest. 

In [21]:
transmissions_sorted = details.sortedTransmissions() # get the sorted transmissions
transmissions_sorted[:6] # only show the first four outputs because the rest is really long to be outputted

[(32.0, [9999]),
 (31.0, [123]),
 (29.0, [758]),
 (27.0, [915, 4939]),
 (25.0, [23, 1996]),
 (24.0, [147, 2584, 6408])]

The way to interpret each tuple is as follows: the 0th element of the tuple will be an integer representing the number of transmissions made. The 1st element will be a list of the numbers of the people who had that many transmissions. For example, for the 0th tuple in the list, 32 transmissions were made by person 9999. But for the 3rd indexed tuple, there were 27 transmissions made by persons 915 & 4939. 